In [3]:
import numpy as np
import tensorflow as tf
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
!pip list

In [4]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [6]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
import cv2
import imghdr

In [8]:
data_dir = "data"

In [9]:
image_exts = ["jpeg", "jpg", "bmp", "png"]

In [14]:
for image_class in os.listdir(data_dir):
    class_path = os.path.join(data_dir, image_class)
    if os.path.isdir(class_path):  # Check if it's a directory
        for image in os.listdir(class_path):
            print(image)

image34.jpeg
image22.jpeg
8-82865_pepsi-can-png-image-background-pepsi-caffeine-free.png
image63.png
1000_F_550205755_88iGCAClLSpKlBkk9W2R1oQYnPEdtjnl.jpg
image59.jpeg
Pepsi-Black-Can.gif
image18.jpeg
image38.jpeg
f2aed73d-6c42-4881-81c3-417f59195b6c.jpg
144-1440330_80-s-pepsi-can-png-png-download-transparent.png
139-1391340_pepsi-diet-33cl-cans-x-24-malta-non.png
pepsi-bottle-png-12.png
pepsi-png-image-22.png
1213123_3-pepsi-swag-can.jpg
giphy_s.gif
gratis-png-pepsi.png
image55.jpeg
beirut-lebanon-april-16-2021-600nw-2048889269.jpg
image43.jpeg
image14.jpeg
pepsi-can-please-stop-hotlinking-pics-page-off-topic-10.png
pepsi_PNG4.png
image15.jpeg
144-1440183_pepsi-can-24-x-330ml-hd-png-download.png
image42.jpeg
image54.jpeg
.DS_Store
pngtree-can-of-pepsi-drink-png-image_10127036.png
image39.jpeg
pngtree-pepsi-produced-in-tyumen-russia-by-pepsico-many-photo-png-image_13418754.png
image19.jpeg
image58.jpeg
image23.jpeg
4c04ef698005f44a_00012000000133_C1N1.png
image35.jpeg
3277619-middle.pn